# Project 1

| Name         | PID          | 
|--------------|--------------|
| Peter Murphy | `petermurphy`|
|Joseph McAlister | `josephrm`|
    
We have neither given nor received unauthorized assistance on this assignment. See the course sylabus for details on the Honor Code policy. In particular, sharing lines of solution code is prohibited.

In [14]:
# imports used 
import pandas as pd
from bs4 import BeautifulSoup as soup
from bs4 import Tag
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import requests
import csv 

# Q1 : Qualitative

## Q1.1: Introduction / Initial Problem Statement
Our goal is to find relationships in suicide rates according to country, age demographic, generation, gdp, and year in roughly the past three decades.

Specifically, some questions we think might be interesting to investigate in depth include:
1. What country has the most suicides, or the highest average rate of suicide from year to year?
2. Is there a relationship between generation or age demographic and propensity for suicide?
3. Is there relationship between a country's gdp and rate of suicide?
4. Do the events of a given year have an impact on global or regionally focused suicides?  If so, which years and what kinds of events?
  - Create our own criteria based on findings, if we can, that contribute to the negation of high suicide rates (e.g. high gdp? positive words in the text analysis as well). As the motivation of the Q.

## Q1.2 Data Collection
Our [primary data set](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016) taken from Kaggle has sufficient information to examine the first 3 hypotheses, but in order to qualify/quantify information about the events from each year which may provide insights into the cause of change in this data from year to year, country to country, etc., we need to collect more data in the hopes of modelling a sentiment value that we can ascribe to each of the years contained within our first data set.

To gather information about each year from 1985 - 2016, we webscrape the Wikipedia pages for each year, saving each of the notable events (see the attached image) into a dataframe which we can further mine for positive/negative sentiment which may give insight into why some years show a global/regional/isolated increase or decrease in suicide rates from year to year or country to country.

![](img1.png)

We observe that each of the relevant bullet points under the `Events` heading on the webpage is stored in a `<ul>` element that directly follows the `<h3>` tag containing a `<span>` describing the month of the year.  With this in mind, our approach is to find the `<span>` tags corresponding to each month, navigate to the next `<ul>` immediately proceeding that `<span>` and gather the text fields from each constituent `<li>` tag within. 

In [69]:
DATA_ALREADY_COLLECTED = False # a flag which can be used to skip the data collection process

In [70]:
if not DATA_ALREADY_COLLECTED:
    years = range(1985, 2017) # the years that correspond to our master data set
    months = ['January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November', 'December']
    base_url = "https://en.wikipedia.org/wiki"

    events = pd.DataFrame()   # stores each event  

    for year in years:        # iterate over each year
        print('.', end='')    # hacky loading bar to indicate progress
        url = f"{base_url}/{year}"

        response = requests.get(url)
        if response.status_code != 200:
            print(f"GET failed with response code: {response.status_code}")
            raise

        year_soup = soup(response.text, 'html5lib')

        for month in months:                             # iterate over each month
            span = year_soup.find('span', {'id': month}) # find the corresponding span
            for elem in span.parent.findNext('ul'):      # iterate over each element in the ul tag that follows
                if isinstance(elem, Tag):                # ensure the element is a Tag (not a NavigableString) 
                    # append the contents of the element to the dataframe
                    events = events.append({'year': int(year), 
                                            'month': month, 
                                            'text': elem.get_text().encode('ascii','ignore').decode()}, 
                                           ignore_index=True)
    events = events[['year', 'month', 'text']]            # reorder columns for clarity
    events['year'] = events['year'].astype('int')         # remove annoying decimal
    # save the data and set the flag so this can be skipped in the future
    with open('data/events.csv', 'w') as f:
        f.write(events.to_csv())
        DATA_ALREADY_COLLECTED = True
else: 
    events = pd.read_csv('data/events.csv')
    events = events.drop('Unnamed: 0', axis=1) # drop the excess index column if preloading

events

................................

,year,month,text
0,1985,January,"January 1 The Internet's Domain Name System is created. Greenland withdraws from the European Economic Community. The first UK cellular mobile phone network is launched by Vodafone. On January 7, Cellnet launches the 2nd UK cellular network."
1,1985,January,"January 7 Japan Aerospace Exploration Agency launches Sakigake, Japan's first interplanetary spacecraft and the first deep space probe to be launched by any country other than the United States or the Soviet Union."
2,1985,January,"January 10 Sinclair C5, the world's first mass-produced velomobile, launched."
3,1985,January,"January 13 A passenger train plunges into a ravine in Ethiopia, killing 428 and injuring over 500 others in the worst railroad disaster in Africa.[1][2]"
4,1985,January,"January 15 Tancredo Neves is elected president of Brazil by the Congress, ending the 21-year military rule."
...,...,...,...
2947,2016,December,"December 19 Andrei Karlov, the Russian ambassador to Turkey, is assassinated by an off-duty Turkish police officer at an art exhibition in Ankara.[52]"
2948,2016,December,"December 22 A study finds the VSV-EBOV vaccine against the Ebola virus between 70 and 100% effective, and thus making it the first proven vaccine against the disease.[53]"
2949,2016,December,"December 23 The United Nations Security Council adopts Resolution 2334 condemning ""Israeli settlements in Palestinian territories occupied since 1967"".[54]"
2950,2016,December,"December 25 2016 Russian Defence Ministry Tupolev Tu-154 crash: A Tupolev Tu-154 jetliner of the Russian Defence Ministry crashes into the Black Sea shortly after taking off from Sochi International Airport, Russia, while en route to Khmeimim Air Base, Syria. All 92 people on board, including 64 members of the Alexandrov Ensemble choir of the Russian Armed Forces, are killed.[55]"


## Q1.3 Data Processing
Next, we want to process and group our data in order to make it easier to analyze our posed questions.

### Q1.3.1 Processing the `Events` Data Set

Beginning with the `events` data set above:
- We clean the data and filter out stop words,
- Tally the number of negative and positive words in each entry using the bag of words approach
  - keep an additional tally of mentions of topical keywords like 'suicide', 'suicidal', etc.
- Add additional measures of sentiment (negative, positive, topical) according to year which we can further join with our primary dataset as needed

In [106]:
# load / define words of negative sentiment
negatives = list(set(open('data/negative.txt').read().split()))
positives = list(set(open('data/positive.txt').read().split()))
stopwords = list(set(open('data/stopwords.txt').read().split()))
keywords = ['suicide', 'suicidal', 'depression'] # define custom topical keywords

def min_max_norm(series):
    return (series - series.min()) / (series.max() - series.min())

# create bag of words/sentiment for each row, 
events_cleaned = events.copy()

# remove unnecessary characters from the text column
events_cleaned['text'] = events_cleaned['text'].apply(lambda t: 
    t.replace(',',' ').replace('.', ' ').replace('\n', ' '))

# store non-stopwords
events_cleaned['bag'] = events_cleaned['text'].apply(lambda w: 
    [w for w in w.lower().split() if w not in stopwords])

# store negative words
events_cleaned['neg_words'] = events_cleaned['bag'].apply(lambda bag: 
    [w for w in bag if w in negatives])

# store positive words
events_cleaned['pos_words'] = events_cleaned['bag'].apply(lambda bag: 
    [w for w in bag if w in positives])

# store topical keywords
events_cleaned['keywords'] = events_cleaned['bag'].apply(lambda bag: 
    [w for w in bag if w in keywords])

# tally number of negatives
events_cleaned['n_neg_words'] = events_cleaned['neg_words'].apply(lambda t: len(t))

# tally number of positives
events_cleaned['n_pos_words'] = events_cleaned['pos_words'].apply(lambda t: len(t))

# tally number of keywords 
events_cleaned['n_keywords'] = events_cleaned['keywords'].apply(lambda t: len(t))

# tally number of total meaningful words
events_cleaned['n_total_words'] = events_cleaned['bag'].apply(lambda t: len(t))

# extract a negative sentiment value 
events_cleaned['neg_sentiment'] = events_cleaned.apply(lambda row: row['n_neg_words']/row['n_total_words'], axis=1)
# normalize neg_sentiment between 0, 1
events_cleaned['neg_sentiment'] = (events_cleaned['neg_sentiment'] - events_cleaned['neg_sentiment'].min()) / (events_cleaned['neg_sentiment'].max() - events_cleaned['neg_sentiment'].min())

# extract a positive sentiment value 
events_cleaned['pos_sentiment'] = events_cleaned.apply(lambda row: row['n_pos_words']/row['n_total_words'], axis=1)
# normalize pos_sentiment between 0, 1
events_cleaned['pos_sentiment'] = (events_cleaned['pos_sentiment'] - events_cleaned['pos_sentiment'].min()) / (events_cleaned['pos_sentiment'].max() - events_cleaned['pos_sentiment'].min())

# extract a topical keyword sentiment value 
events_cleaned['topical_sentiment'] = events_cleaned.apply(lambda row: row['n_keywords']/row['n_total_words'], axis=1)
events_cleaned['topical_sentiment'] = (events_cleaned['topical_sentiment'] - events_cleaned['topical_sentiment'].min()) / (events_cleaned['topical_sentiment'].max() - events_cleaned['topical_sentiment'].min())

events_cleaned.sort_values('pos_sentiment', ascending=False).head()

,year,month,text,bag,neg_words,pos_words,keywords,n_neg_words,n_pos_words,n_keywords,n_total_words,neg_sentiment,pos_sentiment,topical_sentiment
597,1989,November,November 6 The Asia-Pacific Economic Cooperation (APEC) is established,"[november, 6, asia-pacific, economic, cooperation, (apec), established]",[],"[cooperation, established]",[],0,2,0,7,0.000000,1.000000,0.0
1353,1994,July,July 4 Rwandan Patriotic Front troops capture Kigali a major breakthrough in the Rwandan Civil War,"[july, 4, rwandan, patriotic, troops, capture, kigali, major, breakthrough, rwandan, civil, war]",[war],"[patriotic, breakthrough, civil]",[],1,3,0,12,0.291667,0.875000,0.0
1296,1994,January,January 1 The North American Free Trade Agreement (NAFTA) is established The Zapatista Army of National Liberation begins their war in Chiapas Mexico,"[january, 1, north, american, free, trade, agreement, (nafta), established, zapatista, army, national, liberation, begins, war, chiapas, mexico]",[war],"[free, agreement, established, liberation]",[],1,4,0,17,0.205882,0.823529,0.0
523,1989,June,June 22 Ireland's first universities established since independence in 1922 Dublin City University and the University of Limerick open,"[june, 22, ireland's, universities, established, independence, 1922, dublin, city, university, university, limerick, open]",[],"[established, independence, open]",[],0,3,0,13,0.000000,0.807692,0.0
2423,2008,January,January 24 A peace deal is signed in Goma Democratic Republic of the Congo ending the Kivu conflict [6],"[january, 24, peace, deal, signed, goma, democratic, republic, congo, ending, kivu, conflict, [6]]",[conflict],"[peace, deal, democratic]",[],1,3,0,13,0.269231,0.807692,0.0


By sorting our `cleaned_events` data according to the various sentiments, we can see that our naive heuristic of $$\frac{\texttt{# positive, #negaitve, or #topical}}{\texttt{#total}}$$ appropriately measuress the desired characteristics. 

We can examine the top few entries for each to verify.

#### Q1.3.1.1 Events Tagged With Positive Sentiment
Here we can see that each of the most positively rated events _do_ correspond to good things such as:
- establishment of cooperative pacts, agreements, or institutions (APEC, NAFTA, Irish universities)
- breakthroughs in violent conflicts, peace deals.

However, we can also see some areas where our heuristic missassigns some positive scores for conveniently worded, or outright bland events such as:
- "The Islamic Salvation Army the armed wing of the Islamic Salvation Front agrees in principle to disband in Algeria"
- "The Marshall Islands became an associated state under the Compact of Free Association" 

both of which received positive scores due to language which may be have otherwise been ignored or even penalized given the context.

In [120]:
top5 = events_cleaned.sort_values('pos_sentiment', ascending=False).head(10).drop(
    ['bag', 'month','neg_words','keywords', 'n_pos_words', 'n_keywords', 'n_neg_words', 
     'n_total_words', 'topical_sentiment', 'neg_sentiment'], axis=1)
top5

,year,text,pos_words,pos_sentiment
597,1989,November 6 The Asia-Pacific Economic Cooperation (APEC) is established,"[cooperation, established]",1.000000
1353,1994,July 4 Rwandan Patriotic Front troops capture Kigali a major breakthrough in the Rwandan Civil War,"[patriotic, breakthrough, civil]",0.875000
1296,1994,January 1 The North American Free Trade Agreement (NAFTA) is established The Zapatista Army of National Liberation begins their war in Chiapas Mexico,"[free, agreement, established, liberation]",0.823529
523,1989,June 22 Ireland's first universities established since independence in 1922 Dublin City University and the University of Limerick open,"[established, independence, open]",0.807692
2423,2008,January 24 A peace deal is signed in Goma Democratic Republic of the Congo ending the Kivu conflict [6],"[peace, deal, democratic]",0.807692
1860,1999,June 5 The Islamic Salvation Army the armed wing of the Islamic Salvation Front agrees in principle to disband in Algeria,"[salvation, salvation, principle]",0.807692
305,1987,October 3 The CanadaUnited States Free Trade Agreement is reached but still requires ratification This agreement would be a precursor to NAFTA,"[free, agreement, agreement]",0.807692
1578,1996,September 2 A permanent peace agreement is signed at the Malacaan Palace between the Government of the Philippines and the Moro National Liberation Front,"[peace, agreement, liberation]",0.807692
211,1986,October 21 The Marshall Islands became an associated state under the Compact of Free Association,"[compact, free]",0.777778
115,1985,December 8 The South Asian Association for Regional Cooperation (SAARC) is established,"[cooperation, established]",0.777778


#### Q1.3.1.2 Events Tagged With Negative Sentiment
Similarly for negative and topically scored entries, we see generally appropriate events such as:
- acts of war, war crimes, and terror
- disasters, protests, massacres, outbreaks
- suicides

In [125]:
top5 = events_cleaned.sort_values('neg_sentiment', ascending=False).head(10).drop(
    ['bag', 'month','pos_words','keywords', 'n_pos_words','n_neg_words','n_keywords', 
     'n_total_words', 'topical_sentiment', 'pos_sentiment'], axis=1)
top5

,year,text,neg_words,neg_sentiment
528,1989,July 6 The Tel AvivJerusalem bus 405 suicide attack the first Palestinian suicide attack on Israel takes place,"[suicide, attack, suicide, attack]",1.000000
1039,1992,March 3 Turkey's worst coal mine disaster leaves 263 dead near Zonguldak,"[worst, disaster, dead]",0.954545
1259,1993,September 27 War in Abkhazia: Fall of Sukhumi Eduard Shevardnadze accuses Russia of passive complicity,"[war, accuses, passive]",0.875000
260,1987,May 11 Klaus Barbie goes on trial in Lyon for war crimes committed during World War II,"[trial, war, war]",0.875000
2893,2015,December 15 The Islamic Military Counter Terrorism Coalition is formed in order to fight terrorism [75],"[terrorism, fight, terrorism]",0.875000
1072,1992,May 17 Protests begin in Bangkok Thailand against the government of General Suchinda Kraprayoon sparking a bloody crackdown,"[protests, bloody, crackdown]",0.875000
294,1987,August 17 Rudolf Hess is found dead in his cell in Spandau Prison Hess 93 is believed to have committed suicide by hanging himself with an electrical flex He was the last remaining prisoner at the complex which is soon demolished,"[dead, prison, suicide, prisoner, complex]",0.795455
1175,1993,January 24 In Turkey thousands protest the murder of journalist Uur Mumcu,"[protest, murder]",0.777778
698,1990,April 13 Cold War: The Soviet Union apologizes for the Katyn massacre,"[cold, massacre]",0.777778
2690,2012,June 2012 Middle East respiratory syndrome coronavirus outbreak begins,"[syndrome, outbreak]",0.777778


#### Q1.3.1.3 Events Tagged With Topical Sentiment
Something which immediately stands out to us is the significant overlap in topical and negative keywords, namely 'suicide'.  Only one of the top ten events pertaining to our topical keywords actually covers the context that we are concerned with: "March 24 26 In San Diego 39 Heaven's Gate cultists commit mass suicide at their compound." 

This indicates that our topical classification lacks sufficient context to delineate between suicide and suicide bombings. For the sake of our research questions, want to ensure to distinguish between the two and reserve the right to forego use of the topical score heavily in further analysis because of this overlap. This undesired overlap may also be remediated by increasing the topical keywords to incorporate more events that _do_ pertain to our topic. 

In [126]:
top5 = events_cleaned.sort_values('topical_sentiment', ascending=False).head(10).drop(
    ['bag', 'month','pos_words','neg_words', 'n_pos_words','n_neg_words','n_keywords',
     'n_total_words', 'pos_sentiment', 'neg_sentiment'], axis=1)
top5

,year,text,keywords,topical_sentiment
528,1989,July 6 The Tel AvivJerusalem bus 405 suicide attack the first Palestinian suicide attack on Israel takes place,"[suicide, suicide]",1.000000
2402,2007,August 14 Multiple suicide bombings kill 572 people in Qahtaniya northern Iraq [31],[suicide],0.583333
2412,2007,November 6 A suicide bomber kills at least fifty people in Mazar-i-Sharif Afghanistan including six members of the National Assembly,[suicide],0.538462
2524,2009,October 25 Two suicide attacks in Baghdad Iraq kill 155 people and injure at least 721 people [47],[suicide],0.538462
1639,1997,March 24 26 In San Diego 39 Heaven's Gate cultists commit mass suicide at their compound,[suicide],0.538462
2288,2005,July 7 Four coordinated suicide bombings hit central London killing 52 people and injuring over 700 [21],[suicide],0.500000
2283,2005,July 7 Four coordinated suicide bombings hit central London killing 52 people and injuring over 700 [21],[suicide],0.500000
2887,2015,November 12 Two suicide bombers detonated explosives in Bourj el-Barajneh Beirut killing 43 people and injuring over 200 others,[suicide],0.500000
2308,2005,November 9 At least 60 people are killed and 115 more are wounded in a series of coordinated suicide bombings in Amman Jordan [37],[suicide],0.500000
2265,2005,February 14 Former Lebanese Prime Minister Rafic Hariri is assassinated along with 21 others by a suicide bomber in Beirut [5] YouTube is launched,[suicide],0.466667


### Q1.3.2 Processing the Primary Data Set

The primary dataset is formatted as follows:

In [127]:
raw_data = pd.read_csv('data/suicide_rates_master.csv')
raw_data

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z


### Q1.3.1 Processing the Primary Data Set
We assert that other efforts [[1],[2]](#References) have sufficiently researched trends in our main topic of suicide with respect to the `sex` column so we omit for concision.

Additionally, the `country-year` and `HDI` columns can be dropped due to redundancy and lack of sufficinetly complete data, respectively.

In [128]:
data = raw_data.drop(['sex', 'country-year', 'HDI for year'], axis=1)
data

,country,year,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,15-24 years,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,35-54 years,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,15-24 years,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,75+ years,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,25-34 years,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,35-54 years,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,75+ years,9,348465,2.58,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,5-14 years,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,5-14 years,44,2631600,1.67,"63,067,077,179",2309,Generation Z


### Q1.3.2.2 Grouping the Reduced Data 
As it stands, our reduced data can now be grouped in various ways which will make it easier for us to examine our posed questions.

The four questions lend themselves to a variety of groupings including:
1. **by country**: to measure cumulative suicides and regional influences
2. **by generation**: to  gauge shared ~~"struggles"~~ that may transcend regional boundaries
3. **by age range**: for similar reasons as above 
4. **by gdp**: to gauge economic influences
5. **by year**: again to measure cumulative suicides and investigate global/broader regional influences based on the `event` data 


In [129]:
data_by_country = data.groupby('country')
data_by_gen = data.groupby('generation')
data_by_age = data.groupby('age')
data_by_gdp_pc = data.groupby('gdp_per_capita ($)') 
data_by_year = data.groupby('year')
events_by_year = events.groupby('year')

# References
[1] The World Health Organization, [Suicide in the world: Global Health Estimates (booklet)](https://www.who.int/mental_health/prevention/suicide/suicideprevent/en/)

[2] American Foundation for Suicide Prevention, https://afsp.org/suicide-statistics/

### Credits:
Peter - All the beautiful

Joseph - Ideation, Bringing together Suicide Data and year in Reviews idea, Suicide data